Inspired from Bruno Aquino's Kernel:
https://www.kaggle.com/braquino/5-fold-lstm-attention-fully-commented-0-694

I am using early stopping on the 5 Folds Stratified K Folds.

In [49]:
import pandas as pd
import pyarrow.parquet as pq # Used to read the data
import os 
import numpy as np
from keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from keras.models import Model
from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split 
from keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from keras import optimizers # Allow us to access the Adam class to modify some parameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold # Used to use Kfold to train our model
from keras.callbacks import * # This object helps the model to train in a smarter way, avoiding overfitting


In [27]:
# select how many folds will be created
N_SPLITS = 5
# it is just a constant with the measurements data size
sample_size = 800000

In [50]:
# It is the official metric used in this competition
# below is the declaration of a function used inside the keras model, calculation with K (keras backend / thensorflow)
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred = tf.convert_to_tensor(y_pred, np.float32)
    y_true = tf.convert_to_tensor(y_true, np.float32)
    
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [29]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [30]:
# just load train data
df_train = pd.read_csv('../input/metadata_train.csv')
# set index, it makes the data access much faster
df_train = df_train.set_index(['id_measurement', 'phase'])
df_train.head()

signal_id  target
id_measurement phase                   
0              0              0       0
               1              1       0
               2              2       0
1              0              3       1
               1              4       1

In [31]:
# in other notebook I have extracted the min and max values from the train data, the measurements
max_num = 127
min_num = -128

In [32]:
# This function standardize the data from (-128 to 127) to (-1 to 1)
# Theoretically it helps in the NN Model training, but I didn't tested without it
def min_max_transf(ts, min_data, max_data, range_needed=(-1,1)):
    if min_data < 0:
        ts_std = (ts + abs(min_data)) / (max_data + abs(min_data))
    else:
        ts_std = (ts - min_data) / (max_data - min_data)
    if range_needed[0] < 0:    
        return ts_std * (range_needed[1] + abs(range_needed[0])) + range_needed[0]
    else:
        return ts_std * (range_needed[1] - range_needed[0]) + range_needed[0]

In [33]:
def transform_ts(ts, n_dim=160, min_max=(-1,1)):
    # convert data into -1 to 1
    ts_std = min_max_transf(ts, min_data=min_num, max_data=max_num)
    # bucket or chunk size, 5000 in this case (800000 / 160)
    bucket_size = int(sample_size / n_dim)
    # new_ts will be the container of the new data
    new_ts = []
    # this for iteract any chunk/bucket until reach the whole sample_size (800000)
    for i in range(0, sample_size, bucket_size):
        # cut each bucket to ts_range
        ts_range = ts_std[i:i + bucket_size]
        # calculate each feature
        mean = ts_range.mean()
        std = ts_range.std() # standard deviation
        std_top = mean + std # I have to test it more, but is is like a band
        std_bot = mean - std
        # I think that the percentiles are very important, it is like a distribuiton analysis from eath chunk
        percentil_calc = np.percentile(ts_range, [0, 1, 25, 50, 75, 99, 100]) 
        max_range = percentil_calc[-1] - percentil_calc[0] # this is the amplitude of the chunk
        relative_percentile = percentil_calc - mean # maybe it could heap to understand the asymmetry
        new_ts.append(np.concatenate([np.asarray([mean, std, std_top, std_bot, max_range]),percentil_calc, relative_percentile]))
    return np.asarray(new_ts)

In [34]:
# this function take a piece of data and convert using transform_ts(), but it does to each of the 3 phases
# if we would try to do in one time, could exceed the RAM Memmory
def prep_data(start, end):

    praq_train = pq.read_pandas('../input/train.parquet', columns=[str(i) for i in range(start, end)]).to_pandas()
    X = []
    y = []
   
    for id_measurement in tqdm(df_train.index.levels[0].unique()[int(start/3):int(end/3)]):
        X_signal = []
        # for each phase of the signal
        for phase in [0,1,2]:
            # extract from df_train both signal_id and target to compose the new data sets
            signal_id, target = df_train.loc[id_measurement].loc[phase]
            # but just append the target one time, to not triplicate it
            if phase == 0:
                y.append(target)
            # extract and transform data into sets of features
            X_signal.append(transform_ts(praq_train[str(signal_id)]))
        # concatenate all the 3 phases in one matrix
        X_signal = np.concatenate(X_signal, axis=1)
        # add the data to X
        X.append(X_signal)
    X = np.asarray(X)
    y = np.asarray(y)
    return X, y

In [35]:
# this code is very simple, divide the total size of the df_train into two sets and process it
X = []
y = []
def load_all():
    total_size = len(df_train)
    for ini, end in [(0, int(total_size/2)), (int(total_size/2), total_size)]:
        X_temp, y_temp = prep_data(ini, end)
        X.append(X_temp)
        y.append(y_temp)
load_all()
X = np.concatenate(X)
y = np.concatenate(y)

100%|██████████| 1452/1452 [07:04<00:00,  3.48it/s]


In [36]:
print(X.shape, y.shape)
# save data into file, a numpy specific format
np.save("X.npy",X)
np.save("y.npy",y)

(2904, 160, 57) (2904,)


In [37]:
# This is NN LSTM Model creation
def model_lstm(input_shape):
    # The shape was explained above, must have this order
    inp = Input(shape=(input_shape[1], input_shape[2],))
    
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(inp)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = Attention(input_shape[1])(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.25) (x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[matthews_correlation])
    
    return model

In [40]:
# Here is where the training happens

# First, create a set of indexes of the 5 folds
splits = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=2019).split(X, y))
preds_val = []
y_val = []
# Then, iteract with each fold
# If you dont know, enumerate(['a', 'b', 'c']) returns [(0, 'a'), (1, 'b'), (2, 'c')]
for idx, (train_idx, val_idx) in enumerate(splits):
    K.clear_session() # I dont know what it do, but I imagine that it "clear session" :)
    print("Beginning fold {}".format(idx+1))
    train_X, train_y, val_X, val_y = X[train_idx], y[train_idx], X[val_idx], y[val_idx]
    model = model_lstm(train_X.shape)
    es = EarlyStopping(monitor='val_matthews_correlation', verbose=2, patience=15)
    ckpt = ModelCheckpoint('weights_{}.h5'.format(idx), save_best_only=True, save_weights_only=True, verbose=1, monitor='val_matthews_correlation', mode='max')
    # Train, train, train
    model.fit(train_X, train_y, batch_size=128, epochs=50, validation_data=[val_X, val_y], callbacks=[ckpt,es])
    # loads the best weights saved by the checkpoint
    model.load_weights('weights_{}.h5'.format(idx))
    # Add the predictions of the validation to the list preds_val
    preds_val.append(model.predict(val_X, batch_size=512))
    # and the val true y
    y_val.append(val_y)

# concatenates all and prints the shape    
preds_val = np.concatenate(preds_val)[...,0]
y_val = np.concatenate(y_val)
preds_val.shape, y_val.shape

Beginning fold 1
Train on 2322 samples, validate on 582 samples
Epoch 1/50
2322/2322 [==============================] - 5s 2ms/step - loss: 0.3547 - matthews_correlation: 0.0075 - val_loss: 0.2303 - val_matthews_correlation: 0.0000e+00

Epoch 00001: val_matthews_correlation improved from -inf to 0.00000, saving model to weights_0.h5
Epoch 2/50
2322/2322 [==============================] - 3s 1ms/step - loss: 0.2351 - matthews_correlation: 0.0000e+00 - val_loss: 0.2244 - val_matthews_correlation: 0.0000e+00

Epoch 00002: val_matthews_correlation did not improve from 0.00000
Epoch 3/50
2322/2322 [==============================] - 3s 1ms/step - loss: 0.2180 - matthews_correlation: 0.0000e+00 - val_loss: 0.2103 - val_matthews_correlation: 0.0000e+00

Epoch 00003: val_matthews_correlation did not improve from 0.00000
Epoch 4/50
2322/2322 [==============================] - 3s 1ms/step - loss: 0.2149 - matthews_correlation: -0.0014 - val_loss: 0.2022 - val_matthews_correlation: 0.0000e+00

Epo


Epoch 00007: val_matthews_correlation improved from 0.02191 to 0.23348, saving model to weights_1.h5
Epoch 8/50
2323/2323 [==============================] - 3s 1ms/step - loss: 0.1369 - matthews_correlation: 0.3547 - val_loss: 0.1266 - val_matthews_correlation: 0.5305

Epoch 00008: val_matthews_correlation improved from 0.23348 to 0.53051, saving model to weights_1.h5
Epoch 9/50
2323/2323 [==============================] - 3s 1ms/step - loss: 0.1395 - matthews_correlation: 0.5536 - val_loss: 0.1305 - val_matthews_correlation: 0.4950

Epoch 00009: val_matthews_correlation did not improve from 0.53051
Epoch 10/50
2323/2323 [==============================] - 3s 1ms/step - loss: 0.1143 - matthews_correlation: 0.6322 - val_loss: 0.1164 - val_matthews_correlation: 0.6053

Epoch 00010: val_matthews_correlation improved from 0.53051 to 0.60526, saving model to weights_1.h5
Epoch 11/50
2323/2323 [==============================] - 3s 1ms/step - loss: 0.1128 - matthews_correlation: 0.6451 - val_

2323/2323 [==============================] - 3s 1ms/step - loss: 0.1000 - matthews_correlation: 0.6425 - val_loss: 0.1188 - val_matthews_correlation: 0.6101

Epoch 00014: val_matthews_correlation improved from 0.51790 to 0.61010, saving model to weights_2.h5
Epoch 15/50
2323/2323 [==============================] - 3s 1ms/step - loss: 0.1067 - matthews_correlation: 0.6341 - val_loss: 0.1181 - val_matthews_correlation: 0.6477

Epoch 00015: val_matthews_correlation improved from 0.61010 to 0.64769, saving model to weights_2.h5
Epoch 16/50
2323/2323 [==============================] - 3s 1ms/step - loss: 0.1008 - matthews_correlation: 0.7057 - val_loss: 0.1234 - val_matthews_correlation: 0.4433

Epoch 00016: val_matthews_correlation did not improve from 0.64769
Epoch 17/50
2323/2323 [==============================] - 3s 1ms/step - loss: 0.1063 - matthews_correlation: 0.5947 - val_loss: 0.1130 - val_matthews_correlation: 0.6327

Epoch 00017: val_matthews_correlation did not improve from 0.64

2324/2324 [==============================] - 3s 1ms/step - loss: 0.1073 - matthews_correlation: 0.6178 - val_loss: 0.1100 - val_matthews_correlation: 0.5676

Epoch 00021: val_matthews_correlation did not improve from 0.65032
Epoch 22/50
2324/2324 [==============================] - 3s 1ms/step - loss: 0.0973 - matthews_correlation: 0.6740 - val_loss: 0.0999 - val_matthews_correlation: 0.6646

Epoch 00022: val_matthews_correlation improved from 0.65032 to 0.66455, saving model to weights_3.h5
Epoch 23/50
2324/2324 [==============================] - 3s 1ms/step - loss: 0.0970 - matthews_correlation: 0.6350 - val_loss: 0.1047 - val_matthews_correlation: 0.6418

Epoch 00023: val_matthews_correlation did not improve from 0.66455
Epoch 24/50
2324/2324 [==============================] - 3s 1ms/step - loss: 0.0960 - matthews_correlation: 0.6655 - val_loss: 0.1006 - val_matthews_correlation: 0.5133

Epoch 00024: val_matthews_correlation did not improve from 0.66455
Epoch 25/50
2324/2324 [=======

((2904,), (2904,))

In [53]:
import tensorflow as tf

def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)]):
        score = K.eval(matthews_correlation(y_true.astype(np.float64), (y_proba > threshold).astype(np.float64)))
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'matthews_correlation': best_score}
    return search_result

In [54]:
best_threshold = threshold_search(y_val, preds_val)['threshold']

100%|██████████| 100/100 [00:25<00:00,  2.62it/s]


In [55]:
best_threshold

0.5

In [56]:
meta_test = pd.read_csv('../input/metadata_test.csv')

In [57]:
meta_test = meta_test.set_index(['signal_id'])
meta_test.head()

,id_measurement,phase
signal_id,,
8712,2904,0
8713,2904,1
8714,2904,2
8715,2905,0
8716,2905,1


In [58]:
first_sig = meta_test.index[0]
n_parts = 10
max_line = len(meta_test)
part_size = int(max_line / n_parts)
last_part = max_line % n_parts
print(first_sig, n_parts, max_line, part_size, last_part, n_parts * part_size + last_part)
# Here we create a list of lists with start index and end index for each of the 10 parts and one for the last partial part
start_end = [[x, x+part_size] for x in range(first_sig, max_line + first_sig, part_size)]
start_end = start_end[:-1] + [[start_end[-1][0], start_end[-1][0] + last_part]]
print(start_end)
X_test = []
# now, very like we did above with the train data, we convert the test data part by part
# transforming the 3 phases 800000 measurement in matrix (160,57)
for start, end in start_end:
    subset_test = pq.read_pandas('../input/test.parquet', columns=[str(i) for i in range(start, end)]).to_pandas()
    for i in tqdm(subset_test.columns):
        id_measurement, phase = meta_test.loc[int(i)]
        subset_test_col = subset_test[i]
        subset_trans = transform_ts(subset_test_col)
        X_test.append([i, id_measurement, phase, subset_trans])

8712 10 20337 2033 7 20337
[[8712, 10745], [10745, 12778], [12778, 14811], [14811, 16844], [16844, 18877], [18877, 20910], [20910, 22943], [22943, 24976], [24976, 27009], [27009, 29042], [29042, 29049]]


100%|██████████| 7/7 [00:00<00:00,  8.88it/s]


In [59]:
X_test_input = np.asarray([np.concatenate([X_test[i][3],X_test[i+1][3], X_test[i+2][3]], axis=1) for i in range(0,len(X_test), 3)])
np.save("X_test.npy",X_test_input)
X_test_input.shape

(6779, 160, 57)

In [60]:
submission = pd.read_csv('../input/sample_submission.csv')
print(len(submission))
submission.head()

20337


,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [61]:
preds_test = []
for i in range(N_SPLITS):
    model.load_weights('weights_{}.h5'.format(i))
    pred = model.predict(X_test_input, batch_size=300, verbose=1)
    pred_3 = []
    for pred_scalar in pred:
        for i in range(3):
            pred_3.append(pred_scalar)
    preds_test.append(pred_3)


6779/6779 [==============================] - 2s 269us/step


In [63]:
preds_test = (np.squeeze(np.mean(preds_test, axis=0)) > best_threshold).astype(np.int)
preds_test.shape

(20337,)

In [64]:
submission['target'] = preds_test
submission.to_csv('submission.csv', index=False)
submission.head()

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [65]:
preds_test_2 = (np.squeeze(np.max(preds_test, axis=0)) > best_threshold).astype(np.int)
preds_test.shape

(20337,)

In [66]:
submission['target'] = preds_test
submission.to_csv('submission_2.csv', index=False)
submission.head()

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0
